In [1]:
# extract persons and their alt names from judaica link
from SPARQLWrapper import SPARQLWrapper2, XML , RDF , JSON , TURTLE
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, SKOS, OWL, FOAF, RDFS
import os, json, pickle
root_data = "~/jl/data/"
root = "~/jl/"

In [2]:
# prepare entity pages dataset and namespaces
ep = Graph()

foaf = Namespace("http://xmlns.com/foaf/0.1/")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
jl = Namespace("http://data.judaicalink.org/ontology/")
owl = Namespace ("http://www.w3.org/2002/07/owl#")

ep.bind('skos', skos)
ep.bind ('foaf' , foaf)
ep.bind ('jl' , jl)
ep.bind('gndo',gndo)
ep.bind ('owl' , owl)

In [3]:
prefixes = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX gndo: <http://d-nb.info/standards/elementset/gnd#>
PREFIX pro: <http://purl.org/hpi/patchr#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dblp: <http://dblp.org/rdf/schema-2015-01-26#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX bibtex: <http://data.bibbase.org/ontology/#>
PREFIX jl: <http://data.judaicalink.org/ontology/>
"""

In [4]:
sparql = SPARQLWrapper2("http://data.judaicalink.org/sparql/query")
queryString = prefixes + "\n" + "select ?s ?name ?altName {?s a foaf:Person . ?s skos:prefLabel ?name . ?s skos:altLabel ?altName}"
sparql.setQuery(queryString)
sparql.setReturnFormat(TURTLE)
results = sparql.query().fullResult

In [5]:
persons = {}
for hit in results['results']['bindings']:
    uri = hit['s']['value']
    
    if uri not in persons:
        persons[uri] = {'name': '', 'altn': []}
    
    p_name = hit['name']['value']
    p_altn = hit['altName']['value']
    
    persons[uri]['name'] = p_name
    persons[uri]['altn'].append(p_altn)

In [6]:
with open(root_data+"person_names/jl_persons_raw.pickle", "wb")as outfile:
    pickle.dump(persons, outfile)